In [5]:
!pip install transformers
!pip install transformers torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base-mnli')
model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-base-mnli')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00


Some weights of the model checkpoint at microsoft/deberta-base-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
# Add gold label and pairIDs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
labels = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}

gold_labels = pd.read_csv('subset_en.csv', sep='\t')
classified_data = pd.DataFrame(columns=['gold_label', 'English', 'German', 'Spanish', 'Swahili', 'Urdu', 'pairID'])
classified_data['gold_label'] = gold_labels['gold_label']
classified_data['pairID'] = gold_labels['pairID']

# Classify English
data = gold_labels
model.eval()

with torch.no_grad():
    for index, row in data.iterrows():
      #Get needed info
      sentence1 = row['sentence1']
      sentence2 = row['sentence2']

      # Classify
      encoded_input = tokenizer.encode_plus(
        sentence1,
        sentence2,
        padding="max_length",
        truncation='only_first',
        max_length=200,
        return_tensors="pt")
      raw_output = model(**encoded_input).logits
      predicted_label = torch.softmax(raw_output, dim=1).argmax().item()

      # Add label to classidied_data
      classified_data.at[index, 'English'] = labels[predicted_label]

In [20]:
# Classify German to English
data = pd.read_csv('translated_subset_de.tsv', sep='\t')

with torch.no_grad():
    for index, row in data.iterrows():
        # Get needed info
        sentence1 = row['translated_sentence1']
        sentence2 = row['translated_sentence2']

        # Classify
        encoded_input = tokenizer.encode_plus(
          sentence1,
          sentence2,
          padding="max_length",
          truncation='only_first',
          max_length=200,
          return_tensors="pt")
        raw_output = model(**encoded_input).logits
        predicted_label = torch.softmax(raw_output, dim=1).argmax().item()

        # Add label to classidied_data
        classified_data.at[index, 'German'] = labels[predicted_label]

In [21]:
# Classify Spanish to English
data = pd.read_csv('translated_subset_es.tsv', sep='\t')

with torch.no_grad():
    for index, row in data.iterrows():
        # Get needed info
        sentence1 = row['translated_sentence1']
        sentence2 = row['translated_sentence2']

        # Classify
        encoded_input = tokenizer.encode_plus(
          sentence1,
          sentence2,
          padding="max_length",
          truncation='only_first',
          max_length=200,
          return_tensors="pt")
        raw_output = model(**encoded_input).logits
        predicted_label = torch.softmax(raw_output, dim=1).argmax().item()

        # Add label to classidied_data
        classified_data.at[index, 'Spanish'] = labels[predicted_label]

In [22]:
# Classify Swahili to English
data = pd.read_csv('translated_subset_sw.tsv', sep='\t')

with torch.no_grad():
    for index, row in data.iterrows():
        # Get needed info
        sentence1 = row['translated_sentence1']
        sentence2 = row['translated_sentence2']

        # Classify
        encoded_input = tokenizer.encode_plus(
          sentence1,
          sentence2,
          padding="max_length",
          truncation='only_first',
          max_length=200,
          return_tensors="pt")
        raw_output = model(**encoded_input).logits
        predicted_label = torch.softmax(raw_output, dim=1).argmax().item()

        # Add label to classidied_data
        classified_data.at[index, 'Swahili'] = labels[predicted_label]

In [23]:
# Classify Urdu to English
data = pd.read_csv('translated_subset_ur.tsv', sep='\t')

with torch.no_grad():
    for index, row in data.iterrows():
        # Get needed info
        sentence1 = row['translated_sentence1']
        sentence2 = row['translated_sentence2']

        # Classify
        encoded_input = tokenizer.encode_plus(
          sentence1,
          sentence2,
          padding="max_length",
          truncation='only_first',
          max_length=200,
          return_tensors="pt")
        raw_output = model(**encoded_input).logits
        predicted_label = torch.softmax(raw_output, dim=1).argmax().item()

        # Add label to classidied_data
        classified_data.at[index, 'Urdu'] = labels[predicted_label]

# Print translated subsets to file
filename = 'deberta_classifications.tsv'
classified_data.to_csv(filename, sep='\t', index=False)